In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Feature Engineering: credit_card_balance**

A Feature Engineering é um processo fundamental para ciência de dados. Ela se refere ao processo de criação e transformação de variáveis para melhorar o desempenho dos modelos de Machine Learning.
<br>
<br>
Nesse notebook vamos trabalhar na criação de variáveis para o dataset **`credit_card_balance`**. Esse dataset contém informações mensais sobre saldos de cartões de crédito do cliente na PoD Bank.
<br>
<br>
Esses dados revelam padrões de gastos, comportamento de pagamento e tendências de endividamento do cliente ao longo do tempo. Essas informações ajudam os credores a avaliar o risco de inadimplência e a tomar decisões mais precisas sobre a concessão de crédito.
<br>
<br>
Por se tratar de criação de variáveis, nessa etapa geralmente trabalhamos com muito volume de dados e grande demanda de processamento, por essa razão iremos utilizar o **Spark** e **Spark.SQL** para essa etapa.

<br>

## **Pacotes e Bibliotecas**

In [ ]:
# Instalando o PySpark.
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Importando bibliotecas.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
import os
import sys

In [ ]:
# Configurando a sessão Spark.
spark = SparkSession.builder \
    .appName('app_spark') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

<br>
<br>
<br>
<br>

## **Leitura e visualização do conjunto de dados**

In [ ]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
dados = spark.read.csv('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/database/credit_card_balance.csv', header=True, inferSchema=True)
dados.createOrReplaceTempView('dados')

# Mostra os dados.
dados.show()

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+-----------+------------------

In [ ]:
# Checando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
num_rows = dados.count()

# Quantidade de colunas.
num_columns = len(dados.columns)

# Quantidade de IDs únicos.
num_id_prev = dados.select(countDistinct('SK_ID_PREV').alias('num_id_prev')).collect()[0]['num_id_prev']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_PREV" únicos do DataFrame: {num_id_prev}')

Quantidade de linhas do DataFrame: 3840312
Quantidade de colunas do DataFrame: 23
Quantidade de "SK_ID_PREV" únicos do DataFrame: 104307


In [ ]:
# Exibindo a estrutura do schema do DataFrame.
dados.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- AMT_BALANCE: double (nullable = true)
 |-- AMT_CREDIT_LIMIT_ACTUAL: integer (nullable = true)
 |-- AMT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |-- AMT_INST_MIN_REGULARITY: double (nullable = true)
 |-- AMT_PAYMENT_CURRENT: double (nullable = true)
 |-- AMT_PAYMENT_TOTAL_CURRENT: double (nullable = true)
 |-- AMT_RECEIVABLE_PRINCIPAL: double (nullable = true)
 |-- AMT_RECIVABLE: double (nullable = true)
 |-- AMT_TOTAL_RECEIVABLE: double (nullable = true)
 |-- CNT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_CURRENT: integer (nullable = true)
 |-- CNT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |

<br>
<br>
<br>
<br>

## **Criação de flags**

A criação de flags binárias 1 ou 0 será útil para nos auxiliar na visão temporal dos dados e na criação das variáveis.
<br>
<br>
Serão criadas flags para os últimos 3, 6, 12, 24 e 36 meses.

In [ ]:
df_temp_01 = spark.sql('''

SELECT
  *,
  CASE
    WHEN MONTHS_BALANCE >=  -3 THEN 1 ELSE 0 END AS U3M,
  CASE
    WHEN MONTHS_BALANCE >=  -6 THEN 1 ELSE 0 END AS U6M,
  CASE
    WHEN MONTHS_BALANCE >= -12 THEN 1 ELSE 0 END AS U12M,
  CASE
    WHEN MONTHS_BALANCE >= -24 THEN 1 ELSE 0 END AS U24M,
  CASE
    WHEN MONTHS_BALANCE >= -36 THEN 1 ELSE 0 END AS U36M
FROM
  dados
ORDER BY
  `SK_ID_PREV`;

''')
df_temp_01.createOrReplaceTempView('df_temp_01')
df_temp_01.show()

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+---+---+----+----+----+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|U3M|U6M|U12M|U24M|U36M|
+----------+----------

<br>
<br>
<br>
<br>

## **Criação de variáveis explicativas**

Vamos criar as variáveis explicativas com base nas variáveis já existentes, usando a sumarização para as janelas temporais criadas na etapa anterior. Já iremos também agregar na granularidade indivíduo usando o ID **`SK_ID_PREV`**, para que depois a gente possa fazer o join com a tabela de **previous_application**.

In [ ]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  # Verifica se a coluna atual não é uma coluna de flag.
  if not any(flag in coluna for flag in colunas_flags):
    for flag in colunas_flags:
      if 'CNT' in coluna or 'DPD' in coluna:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MAX_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MIN_{coluna.upper()}_{flag.upper()}_CREDIT_CARD'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_02.show(20, False)

Quantidade Total de Variáveis Criadas: 380
Nomes das Variáveis Criadas: ['VL_TOT_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_CREDIT_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_CREDIT_CARD', 'VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_U3M_CREDIT_CARD', 'VL_M

<br>

### **Inserindo colunas de datas**

In [ ]:
# Adicionando as colunas de data ao DataFrame.
df_temp_04 = df_temp_02.withColumn('PK_DATREF', date_format(current_date(), 'yyyyMMdd')) \
                       .withColumn('PK_DAT_PROC', current_date())


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_04.columns
nomes_cols_novas = nomes_cols[1:-2]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_04.columns) - 3)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_04.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_04.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_04.show(20, False)

Quantidade Total de Variáveis Criadas: 380
Nomes das Variáveis Criadas: ['VL_TOT_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U3M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U6M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U12M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U24M_CREDIT_CARD', 'VL_TOT_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_MED_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_MAX_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_MIN_AMT_BALANCE_U36M_CREDIT_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_CREDIT_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_CREDIT_CARD', 'VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_U3M_CREDIT_CARD', 'VL_M

<br>
<br>
<br>
<br>

## **Salvando a tabela sumarizada**

In [ ]:
df_temp_04.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_credit_card_balance')